# Numpy, Sympy, and Matplotlib Tutorial for Fall 2025 ME 314

In [2]:
#IMPORT ALL NECESSARY PACKAGES AT THE TOP OF THE CODE
import sympy as sym
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Markdown, display


In [3]:
# ##############################################################################################
# # If you're using Google Colab, uncomment this section by selecting the whole section and press
# # ctrl+'/' (Linux/Windows) or cmd+'/' (MacOS) on your and keyboard. Run it before you start
# # programming, this will enable the nice LaTeX "display()" function for you. If you're using
# # the local Jupyter environment, leave it alone
# ##############################################################################################


try:
    # Only works in Google Colab
    from google.colab.output._publish import javascript

    def custom_latex_printer(exp, **options):
        url = "https://cdnjs.cloudflare.com/ajax/libs/mathjax/3.1.1/latest.js?config=TeX-AMS_HTML"
        javascript(url=url)
        return sym.printing.latex(exp, **options)

    sym.init_printing(use_latex="mathjax", latex_printer=custom_latex_printer)

except ModuleNotFoundError:
    # Fallback for Jupyter or other environments
    sym.init_printing(use_latex="mathjax")

## Problem 1 (20pts)

Given a function $f(x)=\sin(x)$, find the derivative of $f(x)$ and find the directional derivative of $f(x)$ in the direction $v$.  Moreover, compute these derivatives using Pythons's SymPy package.

In [4]:
####################################
# Part 1: compute derivative of f

# define your symbolic variable here
x = sym.symbols('x')

# define the function f
f = sym.sin(x)

# compute derivative of f
df = f.diff(x)

# output resutls
print("derivative of f: ")
display(df)

####################################
# Part 2: compute directional derivative of f

# define dummy variable epsilon, and the direction v
# note 1: here the character 'r' means raw string
# note 2: here I define the symbol for epsilon with
#          the name "\epsilon", this is for LaTeX printing
#          later. In your case, you can give it any other
#          name you want.
eps, v = sym.symbols(r'\epsilon, v')

# add epsilon into function f
new_f = sym.sin(x + v*eps)

# take derivative of the new function w.r.t. epsilon
df_eps = new_f.diff(eps)

# output this derivative
print("derivative of f wrt eps: ")
display(df_eps)

# now, as you've seen the class, we need evaluate for eps=0 to ...
# ... get the directional derivative. To do this, we need to ...
# ... use SymPy's built-in substitution method "subs()" to ...
# ... replace the epsilon symbol with 0
new_df = df_eps.subs(eps, 0)

# output directional derivative
print("directional derivative of f on v: ")
display(new_df)

derivative of f: 


cos(x)

derivative of f wrt eps: 


v⋅cos(\epsilon⋅v + x)

directional derivative of f on v: 


v⋅cos(x)

**Turn in: A scanned (or photograph from your phone or webcam) copy of your hand written solution for both derivatives (or you can use $\LaTeX$, instead of hand writing). Also, turn in the code used to compute the symbolic solutions for both derivatives and the code output.**

$\begin{aligned}
\frac{d}{dx}\sin(x)
&= \lim_{h \to 0} \frac{\sin(x+h)-\sin(x)}{h}
\qquad\text{(set }h=\varepsilon v(x)\text{)}\\[6pt]
&= \lim_{\varepsilon \to 0} \frac{\sin\!\big(x+\varepsilon v(x)\big)-\sin(x)}
{\varepsilon v(x)}\\[6pt]
&= \lim_{\varepsilon \to 0}
\frac{\sin(x)\cos\!\big(\varepsilon v(x)\big)+\cos(x)\sin\!\big(\varepsilon v(x)\big)-\sin(x)}
{\varepsilon v(x)}
\qquad\text{(use }\sin(a+b)=\sin a\cos b+\cos a\sin b\text{)}\\[6pt]
&= \lim_{\varepsilon \to 0}
\frac{\sin(x)\big(\cos(\varepsilon v(x))-1\big)+\cos(x)\sin(\varepsilon v(x))}
{\varepsilon v(x)}\\[8pt]
&= \lim_{\varepsilon \to 0}\left[
\sin(x)\,\frac{\cos(\varepsilon v(x))-1}{\varepsilon v(x)}
\;+\;
\cos(x)\,\frac{\sin(\varepsilon v(x))}{\varepsilon v(x)}
\right].
\end{aligned}$

Now set $u=\varepsilon v(x)$. As $(\varepsilon\to0)$ we have $(u\to0)$ (provided $(v(x))$ is finite), and we use the standard limits
$(\displaystyle\lim_{u\to0}\frac{\sin u}{u}=1)$ and $(\displaystyle\lim_{u\to0}\frac{\cos u-1}{u}=0)$. Therefore

$\begin{aligned}
\frac{d}{dx}\sin(x)
&= \sin(x)\cdot 0 + \cos(x)\cdot 1\\[4pt]
&= \cos(x).
\end{aligned}$

$
\boxed{\displaystyle
\frac{d}{dx}\sin(x)=\cos(x)
}$


$
\frac{d}{dt}\cos(t)
= \lim_{\varepsilon \to 0}
  \frac{\cos\!\big(t + \varepsilon v(t)\big) - \cos(t)}{\varepsilon v(t)}
$

$
= \lim_{\varepsilon \to 0}
  \frac{\cos(t)\cos\!\big(\varepsilon v(t)\big)
        - \sin(t)\sin\!\big(\varepsilon v(t)\big)
        - \cos(t)}{\varepsilon v(t)}
$

$
= \lim_{\varepsilon \to 0}
  \frac{\cos(t)\big(\cos\!\big(\varepsilon v(t)\big) - 1\big)
        - \sin(t)\sin\!\big(\varepsilon v(t)\big)}{\varepsilon v(t)}
$

$
= \cos(t) \cdot \lim_{\varepsilon \to 0}
    \frac{\cos\!\big(\varepsilon v(t)\big) - 1}{\varepsilon v(t)}
  - \sin(t) \cdot \lim_{\varepsilon \to 0}
    \frac{\sin\!\big(\varepsilon v(t)\big)}{\varepsilon v(t)}
$

$
= \cos(t) \cdot (0) - \sin(t) \cdot (1)
$

$
= -\sin(t)
$


## Problem 2 (20pts)

Given a function of trajectory:
$$
J(x(t)) = \int_0^{\pi/2} \frac{1}{2} x(t)^2 dt
$$
Compute the analytical solution when $x=\cos(t)$, verify your answer by numerical integration.

The code for numerical integration is provided below:

In [5]:
def integrate(func, xspan, step_size):
    '''
    Numerical integration with Euler's method

    Parameters:
    ====================
    func: Python function
        func is the function you want to integrate for
    xspan: list
        xspan is a list of two elements, representing
        the start and end of integration
    step_size:
        a smaller step_size will give a more accurate result

    Returns:
    int_val:
        result of the integration
    ====================
    '''
    x = np.arange(xspan[0], xspan[1], step_size)
    int_val = 0
    for xi in x:
        int_val += func(xi) * step_size
    return int_val

# a simple test
def square(x):
    return x**2

def doCos(x):
    return (0.5) * np.cos(x)**2
#print( integrate(func=square, xspan=[0, 1], step_size=0.01) )

print("Answer is")
print( integrate(func=doCos, xspan=[0, np.pi/2], step_size=0.01) )
# or you just call the function without indicating parameters
# print( integrate(square, [0,1], 0.01) )


Answer is
0.3951990765638034


$\frac{\cos^{2}\left(x\right)}{2}$

$= \frac{1}{2} \, \int \cos^{2}\left(x\right) \, \mathrm{d}x$

Now solving:

$\int \cos^{2}\left(x\right) \, \mathrm{d}x$

$= \int \frac{\cos\left(2x\right) + 1}{2} \, \mathrm{d}x$

$= \frac{1}{2} \, \int \cos\left(2x\right) \, \mathrm{d}x + \frac{1}{2} \, \int 1 \, \mathrm{d}x$

Now solving:

$\int \cos\left(2x\right) \, \mathrm{d}x$

substitute $u=2x$ --> $du=2dx$

$= \frac{1}{2} \, \int \cos\left(u\right) \, \mathrm{d}u$

now solving
$\int \cos\left(u\right) \, \mathrm{d}u$

$= \sin\left(u\right)$

Plug in solved integrals:

$\frac{1}{2} \, \int \cos\left(u\right) \, \mathrm{d}u = \frac{\sin\left(u\right)}{2}
= \frac{\sin\left(2x\right)}{2}$

now solving

$\int 1 \, \mathrm{d}x = x$

Plug in solved integrals:

$\frac{1}{2} \, \int \cos\left(2x\right) \, \mathrm{d}x + \frac{1}{2} \, \int 1 \, \mathrm{d}x$

$= \frac{\sin\left(2x\right)}{4} + \frac{x}{2}$

Plug in solved integrals:

$\frac{1}{2} \, \int \cos^{2}\left(x\right) \, \mathrm{d}x
=  \frac{\sin\left(2x\right)}{8} + \frac{x}{4}$

The problem is solved:

$\int \frac{\cos^{2}\left(x\right)}{2} \, \mathrm{d}x = \frac{\sin\left(2x\right)}{8} + \frac{x}{4} + C$